<a href="https://colab.research.google.com/github/MachineLearnia/Python-Machine-Learning/blob/master/27%20-%20Exploratory%20Data%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etude de cas de classification : Les types spectraux d'étoiles
# Mattias EYHERABIDE

L'objectif de ce sujet d'apprentissage automatique est de déterminer d'après des données de la NASA, un type spectral d'étoile entre : 

- Red Dwarf
- Brown Dwarf
- Main Sequence
- Super Giants
- Hyper Giants

Nous disposons ici de differentes variables de features.
Les noms ont été simplifiés, pour plus de clareté voici à quoi elle correspondent :

- L correspond à la luminosité relative
- R correspond au radius relatif
- A_M correspond à la magnitude absolue (Absolute Magnitude).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pandas_profiling
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
data = pd.read_csv('../input/star-type-classification/Stars.csv')

In [ ]:
data.head()

## Checklist de base
### Analyse de forme : 
- **Variable target** : Type
- **lignes et colonnes** : 240, 7
- **Type de variables** : Majortiée de numérique, le reste sont objet (ici du texte)
- **Analyse des valeurs manquantes** : Aucunes valeurs manquantes


## Exploration des données

#### Détermination de la taille du dataset

In [ ]:
df = data.copy()
df.shape

#### Analyse des type de données

In [ ]:
df.dtypes.value_counts().plot.pie()

#### Recherche de valeurs manquantes

In [ ]:
df.isna().sum()

### Exemen de la colonne target

In [ ]:
df['Type'].value_counts(normalize=True)

### Exemen des features

#### Anayse des types objets

In [ ]:
for col in df.select_dtypes('object'):
    plt.figure()
    df[col].value_counts().plot.bar()

In [ ]:
df['Color'].value_counts(normalize=True)

In [ ]:
df['Spectral_Class'].value_counts(normalize=True)

Nous pouvons voir ici que notre dataset dispose de 17 couleurs différentes avec une majoritée de rouge (46%) ainsi que 7 classes spectrales différentes avec une majoritée de M (46%).

Avec les données de wikipedia nous constatons que les données sont logiques, effectivement une majoritées des étoiles sont des naines rouges (https://fr.wikipedia.org/wiki/Type_spectral#Type_M)

In [ ]:
cor_colSpect = pd.crosstab(df['Color'], df['Spectral_Class'])
cor_colSpect

In [ ]:
cor_colSpect.plot.bar(stacked=True)

Nous pouvons ici voir une forte corellation des données entre la couleur et la classe spectrale.

#### Histogrames des variables continues

In [ ]:
for col in df.select_dtypes('float'):
    plt.figure()
    sns.distplot(df[col])

### Correlation des données

In [ ]:
sns.pairplot(df)

In [ ]:
sns.clustermap(df.corr())

## Relation des données

### Relation Type/Temperature

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x="Type", y="Temperature", data=df)
plt.show

### Relation Type/Luminosité relative

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x="Type", y="L", data=df)
plt.show

### Relation Type/Radius relatif

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x="Type", y="R", data=df)
plt.show

### Relation Type/Magnitude absolue

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x="Type", y="A_M", data=df)
plt.show

### Relation Magnitude Absolue/Luminosité Relative/Radius Relatif

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = Axes3D(fig)
ax.scatter(df["A_M"], df["L"], df["R"], c="green", s=20, alpha=0.5)
plt.show()

### Relation Type/Couleur

In [ ]:
cor_colType = pd.crosstab(df['Color'], df['Type'])

cor_colType.plot.bar(stacked=True)

### Relation Type/Classe Spectrale

In [ ]:
cor_colType = pd.crosstab(df['Spectral_Class'], df['Type'])

cor_colType.plot.bar(stacked=True)

### Conclusion relations

Avec ses observations de relations, nous constatons que nos données sont correlées correctement. Nous pouvons en déduire que les entrainements et estimation de modèles seront assez simple et surtout efficaces.

Du plus, aucuns prétraitement n'est nécessaire.

In [ ]:
pandas_profiling.ProfileReport(df)

## Preprossessing

In [ ]:
objects_columns = ["Color", "Spectral_Class"]

In [ ]:
encodage = LabelEncoder()

for i in objects_columns:
    df[i] = encodage.fit_transform(df[i])

In [ ]:
df.head()

In [ ]:
cor_colSpect = pd.crosstab(df['Color'], df['Spectral_Class'])
cor_colSpect

## Modellisation

In [ ]:
x = df.drop("Type",axis=1)
y = df["Type"]

X_train, X_test, Y_train, Y_test = train_test_split(x,y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=3)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

In [ ]:
def trainModel(model):
    model.fit(X_train, Y_train)
    ypred = model.predict(X_test)
    print(model.score(X_train, Y_train))
    cv = confusion_matrix(Y_test, ypred)
    print(cv)

In [ ]:
trainModel(model)

Notre modèle affiche avec un arbre de décision simple une prédiction de 100% avec une matrice de confusion très convenable, notre modèle est juste.